<a href="https://colab.research.google.com/github/muoyo/chicago-ridesharing/blob/master/notebooks/rideshare_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sodapy

In [16]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
from sodapy import Socrata
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, mean_squared_log_error
%matplotlib inline

In [17]:
def get_random_samples(client, num_samples=200, sample_size=1000, verbose=False):
    
    start = time.time()

    # Perform a $select=count(*) query to determine how large the set is
    results = client.get("m6dm-c72p", select='count(*)' )
    total_rows = int(results[0].get('count', 0))
    row_indices = np.arange(0, total_rows, sample_size)
    results = []

    # Use rand() locally to come up with some offsets
    sample_offsets = np.random.choice(row_indices, size=num_samples, replace=False)


    # Use $limit and $offset in conjunction with a stable $order to pick out individual records. 
    # Ex: $order=facility_id&$limit=1&$offset=<some rand() number>
    for i, offset in enumerate (sample_offsets):

        if verbose:
            print(f'Sample {i}: offset={offset},sample_size={sample_size}')
            print('Pure Python time:', time.time() - start, 'sec.')
        results.extend(client.get("m6dm-c72p", order='trip_id', limit=sample_size, offset=offset, 
                                                  select='''trip_id, trip_start_timestamp, pickup_community_area, fare, tip, trip_total'''))
        
    if verbose:
        print('Pure Python time:', time.time() - start, 'sec.')

    return results

In [18]:
# %run ../python_files/utils
# %run ../python_files/data_cleaning

# df = get_trip_records(100000)
# df = clean_data(df)
# df

In [19]:
# samples = get_random_samples(client, verbose=True )
# samples[:10]

In [20]:
# samples_df = pd.DataFrame.from_records(samples)
# samples_df

In [21]:
client = Socrata('data.cityofchicago.org',
             'Tk6RhuGAFvF9P4ehsysybj3IW',
             username="mokome@gmail.com",
             password="Ch1cago!!")

client.timeout = 10000

results = client.get("m6dm-c72p", limit=100000, select='trip_id, trip_start_timestamp, trip_end_timestamp, trip_seconds, trip_miles, pickup_community_area, dropoff_community_area, fare, tip, additional_charges, trip_total' )

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tip,additional_charges,trip_total
0,31881154328f0e8829fa61215edd2c4f6897bb56,2019-07-19T08:00:00.000,2019-07-19T08:15:00.000,835,2.15573706224,24,8,7.5,0,2.55,10.05
1,3188131b970f8bd6b38f494f9a8513820b784384,2019-09-27T19:30:00.000,2019-09-27T19:30:00.000,536,1.86615116688,28,34,7.5,0,2.55,10.05
2,318813e4563c839011469533c0df7960b2e26ba4,2019-08-22T19:00:00.000,2019-08-22T19:15:00.000,992,3.587869,32,24,10,0,2.55,12.55
3,318814d82a07c53860c4eb08a21ff2403edbe9b5,2019-07-05T13:15:00.000,2019-07-05T13:30:00.000,899,2.863703,32,28,7.5,3,2.55,13.05
4,3188151379849598b97f306682832dd07e425743,2019-07-15T19:30:00.000,2019-07-15T20:00:00.000,1153,10.896428,62,24,15,0,2.55,17.55
...,...,...,...,...,...,...,...,...,...,...,...
99995,3277980a4bf4e6086c2af62f8d56645c811fb0b2,2019-09-22T17:00:00.000,2019-09-22T17:00:00.000,276,1.167083,24,24,5,0,2.55,7.55
99996,3277998c04c3c793114d0d9280d36d29bead28c1,2019-08-10T12:15:00.000,2019-08-10T12:45:00.000,2164,8.445811,28,6,17.5,0,2.55,20.05
99997,327799d0e305c2263129352b9da90c1ae87528bf,2019-08-11T10:15:00.000,2019-08-11T10:45:00.000,1355,8.44646242432,4,32,15,4,2.55,21.55
99998,327799fe68bc9a750e45d5dfddedc834bc37395e,2019-09-11T14:30:00.000,2019-09-11T14:45:00.000,432,2.43254,43,48,5,0,2.55,7.55


In [22]:
columns_to_use = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total' ]

columns_to_drop = [ col for col in results_df.columns if col not in columns_to_use ]
df = results_df.drop(columns=columns_to_drop)

df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])
df['trip_end_timestamp'] = pd.to_datetime(df['trip_end_timestamp'])
df['trip_seconds'] = df['trip_seconds'].fillna('0')
df['trip_seconds'] = df['trip_seconds'].astype('int64')

for col in ['trip_miles', 'fare', 'tip', 'additional_charges', 'trip_total']:
    df[col] = df[col].astype(float) 

df['start_weekday'] = df['trip_start_timestamp'].apply(lambda d: d.weekday())
df['start_hour'] = df['trip_start_timestamp'].apply(lambda d: d.hour)
df['start_time_block'] = df['start_hour'] // 3

df['start_date_plus_hour'] = df['trip_start_timestamp'].apply(lambda d: datetime(d.year, d.month, d.day, d.hour))

weather_df = pd.read_csv('../data/chicago_weather.csv')
weather_df['hour'] = weather_df['hour'].apply(lambda x: '{:02d}'.format(x))
weather_df['start_date_plus_hour'] = pd.to_datetime(weather_df['date'] + ' ' + weather_df['hour'] + ':00:00')
weather_df = weather_df.rename(columns={'icon': 'precip'})

# def set_precip(precip):
#     if precip not in ['rain', 'snow']: 
#         precip = 'clear'

#     return precip

# weather_df['precip'] = weather_df['precip'].apply(set_precip)
precip_df = weather_df[['start_date_plus_hour', 'precip', 'apparentTemperature']]

df = df.merge(precip_df, how='left', on='start_date_plus_hour')
df.head()

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area,fare,tip,additional_charges,trip_total,start_weekday,start_hour,start_time_block,start_date_plus_hour,precip,apparentTemperature
0,31881154328f0e8829fa61215edd2c4f6897bb56,2019-07-19 08:00:00,2019-07-19 08:15:00,835,2.155737,24,7.5,0.0,2.55,10.05,4,8,2,2019-07-19 08:00:00,partly-cloudy-day,87.70
1,3188131b970f8bd6b38f494f9a8513820b784384,2019-09-27 19:30:00,2019-09-27 19:30:00,536,1.866151,28,7.5,0.0,2.55,10.05,4,19,6,2019-09-27 19:00:00,rain,68.38
2,318813e4563c839011469533c0df7960b2e26ba4,2019-08-22 19:00:00,2019-08-22 19:15:00,992,3.587869,32,10.0,0.0,2.55,12.55,3,19,6,2019-08-22 19:00:00,partly-cloudy-day,71.46
3,318814d82a07c53860c4eb08a21ff2403edbe9b5,2019-07-05 13:15:00,2019-07-05 13:30:00,899,2.863703,32,7.5,3.0,2.55,13.05,4,13,4,2019-07-05 13:00:00,clear-day,94.00
4,3188151379849598b97f306682832dd07e425743,2019-07-15 19:30:00,2019-07-15 20:00:00,1153,10.896428,62,15.0,0.0,2.55,17.55,0,19,6,2019-07-15 19:00:00,clear-day,87.16


In [9]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
trip_id                  100000 non-null object
trip_start_timestamp     100000 non-null datetime64[ns]
trip_end_timestamp       100000 non-null datetime64[ns]
trip_seconds             100000 non-null int64
trip_miles               99993 non-null float64
pickup_community_area    93718 non-null object
fare                     100000 non-null float64
tip                      100000 non-null float64
additional_charges       100000 non-null float64
trip_total               100000 non-null float64
start_weekday            100000 non-null int64
start_hour               100000 non-null int64
start_time_block         100000 non-null int64
start_date_plus_hour     100000 non-null datetime64[ns]
precip                   100000 non-null object
apparentTemperature      100000 non-null float64
dtypes: datetime64[ns](3), float64(6), int64(4), object(3)
memory usage: 13.0+ MB


In [23]:
# Limit to the columns we are interested in: 
# 'apparentTemperature', 'start_weekday', 'start_hour', (OR 'start_time_block'), 'pickup_community_area' 

columns_to_use = ['apparentTemperature', 'start_weekday', 'start_hour', 'pickup_community_area']
columns_to_drop = [ col for col in df.columns if col not in columns_to_use ]

# Use dependent variables listed above to predict the independent variable: 'trip_total' OR 'fare'
X = df.drop(columns=columns_to_drop)
y = df['trip_total']


# deal with any null values
X['apparentTemperature']=X['apparentTemperature'].fillna(X['apparentTemperature'].median())
X['pickup_community_area']=X['pickup_community_area'].fillna('0')


# Split out continuous & categorical variables
cont_cols = ['apparentTemperature']
cat_cols = [ col for col in columns_to_use if col not in cont_cols ]

enc = OneHotEncoder()

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

X_train_cont = X_train[cont_cols]
X_train_cat = X_train[cat_cols]

# ss = StandardScaler()
# X_train_cont = pd.DataFrame(ss.fit_transform(X_train_cont))

# Transform training set
X_train_enc = enc.fit_transform(X_train_cat, y_train)

# Convert these columns into a DataFrame 
columns = enc.get_feature_names(input_features=X_train_cat.columns)
X_train_cat = pd.DataFrame(X_train_enc.todense(), columns=columns, index=X_train.index)

# Combine categorical and continuous features into the final dataframe
X_train = pd.concat([X_train_cont, X_train_cat], axis=1)
X_train_const = sm.add_constant(X_train)

# Fit model & show summary
model = sm.OLS(y_train,X_train_const).fit()
model.summary()

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             trip_total   R-squared:                       0.285
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     278.5
Date:                Fri, 24 Jan 2020   Prob (F-statistic):               0.00
Time:                        21:46:50   Log-Likelihood:            -2.7861e+05
No. Observations:               75000   AIC:                         5.574e+05
Df Residuals:                   74892   BIC:                         5.584e+05
Df Model:                         107                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         12.6649      0.321     39.490      0.000      12.036      13.293
apparentTemperature           -0.0058      0.005     -1.165      0.244      -0.016       0.004
start_weekday_0.0              1.7648      0.107     16.430      0.000       1.554       1.975
start_weekday_1.0              1.6297      0.108     15.101      0.000       1.418       1.841
start_weekday_2.0              1.6384      0.106     15.508      0.000       1.431       1.845
start_weekday_3.0              2.1002      0.099     21.300      0.000       1.907       2.293
start_weekday_4.0              1.9073      0.097     19.669      0.000       1.717       2.097
start_weekday_5.0              1.7076      0.093     18.285      0.000       1.525       1.891
start_weekday_6.0              1.9170      0.100     19.198      0.000       1.721       2.113
start_hour_0.0                -0.5876      0.189     -3.104      0.002      -0.959      -0.217
start_hour_1.0                -1.3964      0.228     -6.129      0.000      -1.843      -0.950
start_hour_2.0                -1.2298      0.269     -4.566      0.000      -1.758      -0.702
start_hour_3.0                 0.9273      0.327      2.831      0.005       0.285       1.569
start_hour_4.0                 4.1216      0.325     12.697      0.000       3.485       4.758
start_hour_5.0                 5.1205      0.279     18.325      0.000       4.573       5.668
start_hour_6.0                 3.3982      0.228     14.887      0.000       2.951       3.846
start_hour_7.0                 1.2175      0.190      6.423      0.000       0.846       1.589
start_hour_8.0                 0.2605      0.166      1.571      0.116      -0.065       0.586
start_hour_9.0                -0.0444      0.170     -0.262      0.794      -0.377       0.288
start_hour_10.0                0.4585      0.174      2.630      0.009       0.117       0.800
start_hour_11.0                1.1045      0.171      6.459      0.000       0.769       1.440
start_hour_12.0                0.9116      0.170      5.367      0.000       0.579       1.245
start_hour_13.0                0.4939      0.171      2.884      0.004       0.158       0.830
start_hour_14.0                1.0678      0.172      6.197      0.000       0.730       1.406
start_hour_15.0                1.1678      0.167      7.010      0.000       0.841       1.494
start_hour_16.0                0.9741      0.159      6.120      0.000       0.662       1.286
start_hour_17.0                0.8007      0.150      5.338      0.000       0.507       1.095
start_hour_18.0               -0.6495      0.147     -4.427      0.000      -0.937      -0.362
start_hour_19.0               -0.7773      0.148     -5.246      0.000      -1.068      -0.487
start_hour_20.0               -1.1397      

In [69]:
# %run ../python_files/regression

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = get_train_test_split(df, test_size=0.25)
# X_train_const = sm.add_constant(X_train)

# # Fit model & show summary
# OLS(y_train,X_train_const)


In [70]:
# LinearRegression(X_train, y_train)

In [24]:
# Fit the model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Print R2 and MSE for training set
print('Training r^2:', linreg.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, linreg.predict(X_train)))

linreg.coef_

Training r^2: 0.28466690677623874
Training MSE: 98.67310294092059


array([-5.84083739e-03, -3.64429837e+11, -3.64429837e+11, -3.64429837e+11,
       -3.64429837e+11, -3.64429837e+11, -3.64429837e+11, -3.64429837e+11,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
        8.24379735e+10,  8.24379735e+10,  8.24379735e+10,  8.24379735e+10,
       -7.47424596e+09, -7.47424598e+09, -7.47424598e+09, -7.47424598e+09,
       -7.47424598e+09, -7.47424598e+09, -7.47424598e+09, -7.47424598e+09,
       -7.47424598e+09, -7.47424598e+09, -7.47424598e+09, -7.47424598e+09,
       -7.47424598e+09, -7.47424598e+09, -7.47424598e+09, -7.47424598e+09,
       -7.47424598e+09, -7.47424598e+09, -7.47424598e+09, -7.47424598e+09,
       -7.47424598e+09, -

In [25]:
Lasso(X_train, y_train)

Lasso(alpha=       apparentTemperature  start_weekday_0.0  start_weekday_1.0  \
21577                68.66                0.0                0.0   
70625                72.59                0.0                1.0   
69444                71.64                0.0                0.0   
6894                 89.75                0.0                0.0   
9930                 82.18                0.0                0.0   
...                    ...                ...                ...   
70541                78.05                0.0                0.0   
26891                76.60                0.0                0.0   
38356                66.95                0.0                0.0   
46551                73.59                0.0                0.0   
42496               105.96                0.0                0.0   

       start_weekday_2.0  start_weekday_3.0  start_weekday_4.0  \
21577                0.0                0.0                0.0   
70625                0.0               

In [34]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1) # Lasso is also known as the L1 norm 
lasso.fit(X_train, y_train)

print('Training r^2:', lasso.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, lasso.predict(X_train)))

lasso.coef_

Training r^2: 0.0016563287686486294
Training MSE: 158.05216084339293


array([ 0.03126329,  0.        , -0.        ,  0.        ,  0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        , -0.        ,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        ,  0.        , -0.  

In [40]:
Ridge(X_train, y_train)

Training r^2: 0.24422052753280088
Training MSE: 119.65075974006857


Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [29]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=10) # Ridge is also known as the L2 norm
ridge.fit(X_train, y_train)

print('Training r^2:', ridge.score(X_train, y_train))
print('Training MSE:', mean_squared_error(y_train, ridge.predict(X_train)))

ridge.coef_

Training r^2: 0.24422052753280088
Training MSE: 119.65075974006857


array([ 0.03425152,  0.13272144, -0.57440017, -0.18399975, -0.05173743,
        0.43186143,  0.07358853,  0.17196594, -1.65422202,  4.28157632,
        1.4762323 , -0.66009259, -0.23871675,  0.41103615, -1.91738105,
       -1.69843236, 12.76272151, -0.3584145 , -1.66420566, -0.68058555,
       -0.464841  , -0.2946133 ,  0.34010065, -1.02033888,  0.57019683,
        0.15088632,  0.6604779 , -0.09463474,  0.9029411 , -0.65924804,
       -1.20456053, -0.26349371, -1.07032985, -0.74448602, -0.67617293,
       -2.0822037 , -1.54646591, -1.53596207, -1.1292946 , -2.17275856,
       -1.58137192, -1.65245353, -1.59816161, -2.28320311, -0.9291831 ,
       -2.15192645, -1.41643966, -1.66409653,  1.25574547,  2.62983841,
        0.97694035, -2.36086833, -0.91289014, -3.28332547, -1.3715935 ,
       -0.401022  , -2.73603591,  1.07161873, -0.4308393 , -0.47457522,
       -1.78048253, -2.205615  , -0.78472142,  0.88219358,  0.4729512 ,
       -1.80458952, -1.11047284,  1.33968932,  6.56498372,  0.75